In [1]:
from pasteur.kedro.ipython import * # type: ignore
register_kedro() # type: ignore

[17:59:14] WARNING  Disabling dataset adult, path "/home/dev/Desktop/pasteur/raw/adult" doesn't exist.                                  main.py:59
           WARNING  Disabling dataset mimic, path "/home/dev/Desktop/pasteur/raw/mimiciv_1_0" doesn't exist.                            main.py:59
           WARNING  Disabling tab_adult, missing dataset adult.                                                                         main.py:69
           WARNING  Disabling mimic_tab_admissions, missing dataset mimic.                                                              main.py:69
[17:59:15] INFO     Kedro project Pasteur                                                                                           __init__.py:77
           INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'                                         __init__.py:78
[17:59:17] WARNING  /home/dev/Desktop/pasteur/venv/lib/python3.10/site-packages/plotly/graph_objects/__init__.py:288: 

In [2]:
%load_ext autoreload
%autoreload 2

           WARNING  /home/dev/Desktop/pasteur/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py:121:              warnings.py:109
                    see the module's documentation for alternative uses                                                                           
                      from imp import reload                                                                                                      
                                                                                                                                                  


In [3]:
view = "texas_charges"

# LIM = 100000
enc = catalog.load(f'{view}.wrk.idx_table').sample()#[:LIM]
trn = catalog.load(f'{view}.trn.table')
attrs = trn['idx'].get_attributes()
len(enc)

9089378


In [4]:
from pasteur.extras.synth.math import expand_table, calc_marginal, AttrSelector, AttrSelectors, get_dtype

cols, cols_noncommon, domains = expand_table(attrs, enc)
domains.keys()

dict_keys(['revenue_code', 'hcpcs_qualifier', 'hcpcs_procedure_code', 'modifier_1', 'modifier_2', 'modifier_3', 'modifier_4', 'unit_measurement_code', 'units_of_service', 'unit_rate', 'chrgs_line_item', 'chrgs_non_cov'])


In [5]:
tst = ['unit_measurement_code', 'unit_measurement_code', 'unit_measurement_code', 'unit_measurement_code','unit_measurement_code','unit_measurement_code']
# tst = ['unit_measurement_code', 'unit_measurement_code', 'unit_measurement_code']

In [6]:
x = AttrSelector(tst[0], 0, {tst[0]: 0})
p = {str(i): AttrSelector(col, 0, {col: 0}) for i, col in enumerate(tst[1:])}

%timeit calc_marginal(cols, cols_noncommon, domains, x, p)

210 ms ± 3.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
import numpy as np

%timeit np.histogramdd(enc[tst].to_numpy(), [domains[col][0] for col in tst])

2.56 s ± 59.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%load_ext line_profiler

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_87858/539894476.py:1 in <cell line: 1>                                            │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_87858/539894476.py'                         │
│                                                                                                  │
│ /home/dev/Desktop/pasteur/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:241 │
│ 8 in run_line_magic                                                                              │
│                                                                                                  │
│   2415 │   │   │   if getattr(fn, "needs_local_scope", False):                                   │
│   2416 │   │   │   │   kwargs['local_ns'] = self.get_local_scope(stack_depth)                    │
│   2417 │   │   │   with self.builtin_trap:                                                       │
│ ❱ 2418 │   │   │   │   result = fn(*args, **kwargs)                                              │
│   2419 │   │   │   return result                                                                 │
│   2420 │                                                                                         │
│   2421 │   def get_local_scope(self, stack_depth):                                               │
│                                                                                                  │
│ /home/dev/Desktop/pasteur/venv/lib/python3.10/site-packages/decorator.py:232 in fun              │
│                                                                                                  │
│   229 │   │   def fun(*args, **kw):                                                              │
│   230 │   │   │   if not kwsyntax:                                                               │
│   231 │   │   │   │   args, kw = fix(args, kw, sig)                                              │
│ ❱ 232 │   │   │   return caller(func, *(extras + args), **kw)                                    │
│   233 │   fun.__name__ = func.__name__                                                           │
│   234 │   fun.__doc__ = func.__doc__                                                             │
│   235 │   fun.__wrapped__ = func                                                                 │
│                                                                                                  │
│ /home/dev/Desktop/pasteur/venv/lib/python3.10/site-packages/IPython/core/magic.py:187 in         │
│ <lambda>                                                                                         │
│                                                                                                  │
│   184 │   # This is a closure to capture the magic_kind.  We could also use a class,             │
│   185 │   # but it's overkill for just that one bit of state.                                    │
│   186 │   def magic_deco(arg):                                                                   │
│ ❱ 187 │   │   call = lambda f, *a, **k: f(*a, **k)                                               │
│   188 │   │                                                                                      │
│   189 │   │   if callable(arg):                                                                  │
│   190 │   │   │   # "Naked" decorator call (just @foo, no args)                                  │
│                                                                                                  │
│ /home/dev/Desktop/pasteur/venv/lib/python3.10/site-packages/IPython/core/magics/extension.py:33  │
│ in load_ext                                                                                      │
│                                                            

In [9]:

def calc_marginal2(
    cols: dict[str, list[np.ndarray]],
    cols_noncommon: dict[str, list[np.ndarray]],
    domains: dict[str, list[int]],
    x: AttrSelector,
    p: AttrSelectors,
    partial: bool = False,
    zero_fill: float | None = None,
):
    """Calculates the 1 way and 2 way marginals between the subsection of the
    hierarchical attribute x and the attributes p(arents)."""

    # Find integer dtype based on domain
    p_dom = 1
    for attr in p.values():
        common = attr.common
        l_dom = 1
        for i, (n, h) in enumerate(attr.cols.items()):
            l_dom *= domains[n][h] - common
        p_dom *= l_dom + common
    x_dom = 1
    for i, (n, h) in enumerate(x.cols.items()):
        x_dom *= domains[n][h] - x.common
    x_dom += x.common

    dtype = get_dtype(p_dom * x_dom)

    n = len(next(iter(cols.values()))[0])
    _sum_nd = np.zeros((n,), dtype=dtype)
    _tmp_nd = np.zeros((n,), dtype=dtype)

    # Handle parents
    mul = 1
    for attr_name, attr in p.items():
        common = attr.common
        l_mul = 1
        p_partial = partial and attr_name == x.name
        for i, (n, h) in enumerate(attr.cols.items()):
            if common == 0 or i == 0:
                np.multiply(cols[n][h], mul * l_mul, out=_tmp_nd, dtype=dtype)
            else:
                np.multiply(cols_noncommon[n][h], mul * l_mul, out=_tmp_nd, dtype=dtype)

            np.add(_sum_nd, _tmp_nd, out=_sum_nd, dtype=dtype)
            l_mul *= domains[n][h] - common

        if p_partial:
            mul *= l_mul
        else:
            mul *= l_mul + common

    # Handle x
    common = x.common
    for i, (n, h) in enumerate(x.cols.items()):
        if common == 0 or (i == 0 and not partial):
            np.multiply(cols[n][h], mul, out=_tmp_nd, dtype=dtype)
        else:
            np.multiply(cols_noncommon[n][h], mul, out=_tmp_nd, dtype=dtype)

        np.add(_sum_nd, _tmp_nd, out=_sum_nd, dtype=dtype)
        mul *= domains[n][h] - common

    # Keep only non-common items if there is a parent to source the others
    if partial:
        n = next(iter(x.cols))
        _sum_nd = _sum_nd[cols[n][0] >= common]
        x_dom = x_dom - x.common

    return np.bincount(_sum_nd, minlength=p_dom * x_dom)#.reshape(x_dom, p_dom)
    # return np.zeros()

    # j_mar = margin
    # x_mar = np.sum(margin, axis=1)
    # p_mar = np.sum(margin, axis=0)

    # return j_mar, x_mar, p_mar

%timeit calc_marginal2(cols, cols_noncommon, domains, x, p)
# %lprun -f calc_marginal2 calc_marginal2(cols, cols_noncommon, domains, x, p)
print(calc_marginal2(cols, cols_noncommon, domains, x, p)[:10])

224 ms ± 6.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[460   0   0   0   0   0   0   0   0   0]


In [10]:
import jax.numpy as jnp
from jax import grad, jit, vmap, default_device, devices
from jax import random, ops
from typing import NamedTuple
import jax.lax as lax


class JaxSelector(NamedTuple):
    common: int
    cols: list[tuple[int, jnp.ndarray]]


JaxSelectors = list[JaxSelector]


def collapse_marginal(x: JaxSelector, P: JaxSelectors):
    # Handle parents
    sum_arr = None
    mul = 1
    for p in P:
        common = p.common
        l_mul = 1
        for dom, col in p.cols:
            tmp_arr = col * (mul * l_mul)
            if sum_arr is None:
                sum_arr = tmp_arr
            else:
                sum_arr += tmp_arr
            l_mul *= dom - common

    # Handle x
    assert sum_arr is not None
    common = x.common
    for dom, col in x.cols:
        sum_arr += dom * mul
        mul *= dom - common

    return sum_arr

In [23]:

def count(sum_arr, length: int):
    # return jnp.bincount(sum_arr, length=length)
    # return lax.broadcast(jnp.uint32(0), [length]).at[sum_arr].add(jnp.uint8(1))
    op = lax.broadcast(jnp.uint16(0), [length])
    sc = lax.expand_dims(sum_arr, [1])
    up = lax.broadcast(jnp.uint16(1), [sc.shape[0]])
    return lax.scatter_add(op, sc, up, lax.ScatterDimensionNumbers(tuple(), (0,), (0,)), mode="promise_in_bounds")

def fun():
    x = JaxSelector(0, [(domains[tst[0]][0], jnp.array(cols[tst[0]][0]))])
    P = []
    for col in tst[1:]:
        P.append(JaxSelector(0, [(domains[col][0], jnp.array(cols[col][0]))]))
    
    p_dom = 1
    for p in P:
        common = p.common
        l_dom = 1
        for dom, _ in p.cols:
            l_dom *= dom - common
        p_dom *= l_dom + common
    x_dom = 1
    for dom, _ in x.cols:
        x_dom *= dom - x.common
    x_dom += x.common
    length = p_dom * x_dom
    print(length)

    try:
        cmp = jit(collapse_marginal).lower(x, P)
        # print(cmp.as_text())
        cmp = cmp.compile()
        # print(cmp.as_text())

        print(jit(count, static_argnums=1).lower(cmp(x, P), length).compile().as_text())
        print(count(cmp(x, P), length)[:10])
    except Exception as e:
        print(e)
        return

    TIMES = 10
    def xn():
        out = count(cmp(x, P), length)

        for i in range(TIMES-1):
            out += count(cmp(x, P), length)
        
        out.block_until_ready()

    # %timeit xn()
    %timeit count(cmp(x, P), length).block_until_ready()
    %timeit xn()


for cuda in (False,): #True):
    d = devices('cuda' if cuda else 'cpu')[0]
    with default_device(d):
        fun()


46656
HloModule jit_count, entry_computation_layout={(u8[9089378]{0})->u16[46656]{0}}

%fused_computation (param_0: u16[46656], param_1.3: u16[9089378], param_2.3: s32[], param_3.6: u8[9089378]) -> u16[46656] {
  %param_0 = u16[46656]{0} parameter(0)
  %constant.17 = u8[1]{0} constant({63})
  %param_3.6 = u8[9089378]{0} parameter(3)
  %param_2.3 = s32[] parameter(2)
  %dynamic-slice.5 = u8[1]{0} dynamic-slice(u8[9089378]{0} %param_3.6, s32[] %param_2.3), dynamic_slice_sizes={1}
  %compare.3 = pred[1]{0} compare(u8[1]{0} %constant.17, u8[1]{0} %dynamic-slice.5), direction=GE
  %reshape.22 = u8[] reshape(u8[1]{0} %dynamic-slice.5)
  %dynamic-slice.4 = u16[1]{0} dynamic-slice(u16[46656]{0} %param_0, u8[] %reshape.22), dynamic_slice_sizes={1}
  %param_1.3 = u16[9089378]{0} parameter(1)
  %dynamic-slice.3 = u16[1]{0} dynamic-slice(u16[9089378]{0} %param_1.3, s32[] %param_2.3), dynamic_slice_sizes={1}
  %add.2 = u16[1]{0} add(u16[1]{0} %dynamic-slice.4, u16[1]{0} %dynamic-slice.3), metadata=

In [12]:
%load_ext cython

In [13]:
%%cython -a
import cython as cy

import numpy as np
cimport numpy as np

def numba_inner_6(
    dom: int, l: int, out: np.ndarray, m1, a1, m2, a2, m3, a3, m4, a4, m5, a5, m6, a6
):
    if not out:
        out = np.zeros((dom,), dtype=np.uint32)

    for i in range(l):
        idx = m1*a1[i] + m2*a2[i] + m3*a3[i] + m4*a4[i] + m5*a5[i] + m6*a6[i]
        # idx = 0

        out[idx] += 1

    return out


In file included from /home/dev/Desktop/pasteur/venv/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1948,
                 from /home/dev/Desktop/pasteur/venv/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/dev/Desktop/pasteur/venv/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /home/dev/.cache/ipython/cython/_cython_magic_1515be3ec9750f784fb0a3988890328e.c:769:
/home/dev/Desktop/pasteur/venv/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~


<!DOCTYPE html>
<!-- Generated by Cython 0.29.32 -->
 
 
 
 Cython: _cython_magic_1515be3ec9750f784fb0a3988890328e.pyx 
 
 
 
 Generated by Cython 0.29.32 
 
 Yellow lines hint at Python interaction. 
 Click on a line that starts with a " + " to see the C code that Cython generated for it.
 
 + 01 : import cython as cy 
 __pyx_t_1 = __Pyx_PyDict_NewPresized (0); if (unlikely(!__pyx_t_1)) __PYX_ERR(0, 1, __pyx_L1_error) 
 __Pyx_GOTREF (__pyx_t_1);
 if ( PyDict_SetItem (__pyx_d, __pyx_n_s_test, __pyx_t_1) < 0) __PYX_ERR(0, 1, __pyx_L1_error) 
 __Pyx_DECREF (__pyx_t_1); __pyx_t_1 = 0;
   02 : 
 + 03 : import numpy as np 
 __pyx_t_1 = __Pyx_Import (__pyx_n_s_numpy, 0, 0); if (unlikely(!__pyx_t_1)) __PYX_ERR(0, 3, __pyx_L1_error) 
 __Pyx_GOTREF (__pyx_t_1);
 if ( PyDict_SetItem (__pyx_d, __pyx_n_s_np, __pyx_t_1) < 0) __PYX_ERR(0, 3, __pyx_L1_error) 
 __Pyx_DECREF (__pyx_t_1); __pyx_t_1 = 0;
   04 : cimport numpy as np 
   05 : 
 + 06 : def numba_inner_6 ( 
 /* Python wrapper */
static PyObject *__pyx_pw_46_cython_magic_1515be3ec9750f784fb0a3988890328e_1numba_inner_6(PyObject *__pyx_self, PyObject *__pyx_args, PyObject *__pyx_kwds); /*proto*/
static PyMethodDef __pyx_mdef_46_cython_magic_1515be3ec9750f784fb0a3988890328e_1numba_inner_6 = {"numba_inner_6", (PyCFunction)(void*)(PyCFunctionWithKeywords)__pyx_pw_46_cython_magic_1515be3ec9750f784fb0a3988890328e_1numba_inner_6, METH_VARARGS|METH_KEYWORDS, 0};
static PyObject *__pyx_pw_46_cython_magic_1515be3ec9750f784fb0a3988890328e_1numba_inner_6(PyObject *__pyx_self, PyObject *__pyx_args, PyObject *__pyx_kwds) {
 PyObject *__pyx_v_dom = 0;
 PyObject *__pyx_v_l = 0;
 PyArrayObject *__pyx_v_out = 0;
 PyObject *__pyx_v_m1 = 0;
 PyObject *__pyx_v_a1 = 0;
 PyObject *__pyx_v_m2 = 0;
 PyObject *__pyx_v_a2 = 0;
 PyObject *__pyx_v_m3 = 0;
 PyObject *__pyx_v_a3 = 0;
 PyObject *__pyx_v_m4 = 0;
 PyObject *__pyx_v_a4 = 0;
 PyObject *__pyx_v_m5 = 0;
 PyObject *__pyx_v_a5 = 0;
 PyObject *__pyx_v_m6 = 0;
 PyObject *__pyx_v_a6 = 0;
 PyObject *__pyx_r = 0;
 __Pyx_RefNannyDeclarations 
 __Pyx_RefNannySetupContext ("numba_inner_6 (wrapper)", 0);
 {
 static PyObject **__pyx_pyargnames[] = {&__pyx_n_s_dom,&__pyx_n_s_l,&__pyx_n_s_out,&__pyx_n_s_m1,&__pyx_n_s_a1,&__pyx_n_s_m2,&__pyx_n_s_a2,&__pyx_n_s_m3,&__pyx_n_s_a3,&__pyx_n_s_m4,&__pyx_n_s_a4,&__pyx_n_s_m5,&__pyx_n_s_a5,&__pyx_n_s_m6,&__pyx_n_s_a6,0};
 PyObject* values[15] = {0,0,0,0,0,0,0,0,0,0,0,0,0,0,0};
 if (unlikely(__pyx_kwds)) {
 Py_ssize_t kw_args;
 const Py_ssize_t pos_args = PyTuple_GET_SIZE (__pyx_args);
 switch (pos_args) {
 case 15: values[14] = PyTuple_GET_ITEM (__pyx_args, 14);
 CYTHON_FALLTHROUGH;
 case 14: values[13] = PyTuple_GET_ITEM (__pyx_args, 13);
 CYTHON_FALLTHROUGH;
 case 13: values[12] = PyTuple_GET_ITEM (__pyx_args, 12);
 CYTHON_FALLTHROUGH;
 case 12: values[11] = PyTuple_GET_ITEM (__pyx_args, 11);
 CYTHON_FALLTHROUGH;
 case 11: values[10] = PyTuple_GET_ITEM (__pyx_args, 10);
 CYTHON_FALLTHROUGH;
 case 10: values[9] = PyTuple_GET_ITEM (__pyx_args, 9);
 CYTHON_FALLTHROUGH;
 case 9: values[8] = PyTuple_GET_ITEM (__pyx_args, 8);
 CYTHON_FALLTHROUGH;
 case 8: values[7] = PyTuple_GET_ITEM (__pyx_args, 7);
 CYTHON_FALLTHROUGH;
 case 7: values[6] = PyTuple_GET_ITEM (__pyx_args, 6);
 CYTHON_FALLTHROUGH;
 case 6: values[5] = PyTuple_GET_ITEM (__pyx_args, 5);
 CYTHON_FALLTHROUGH;
 case 5: values[4] = PyTuple_GET_ITEM (__pyx_args, 4);
 CYTHON_FALLTHROUGH;
 case 4: values[3] = PyTuple_GET_ITEM (__pyx_args, 3);
 CYTHON_FALLTHROUGH;
 case 3: values[2] = PyTuple_GET_ITEM (__pyx_args, 2);
 CYTHON_FALLTHROUGH;
 case 2: values[1] = PyTuple_GET_ITEM (__pyx_args, 1);
 CYTHON_FALLTHROUGH;
 case 1: values[0] = PyTuple_GET_ITEM (__pyx_args, 0);
 CYTHON_FALLTHROUGH;
 case 0: break;
 default: goto __pyx_L5_argtuple_error;
 }
 kw_args = PyDict_Size (__pyx_kwds);
 switch (pos_args) {
 case 0:
 if (likely((values[0] = __Pyx_PyDict_GetItemStr (__pyx_kwds, __pyx_n_s_dom)) != 0)) kw_args--;
 else goto __pyx_L5_argtuple_error;
 CYTHON_FALLTHROUGH;
 case 1:
 if (li

In [14]:
def numba_inner(dom: int, l: int, out: np.ndarray, *o: Op):
    assert len(o) == 6, f'Length is {len(o)}'
    return numba_inner_6(
        dom,
        l,
        out,
        o[0][0],
        o[0][1],
        o[1][0],
        o[1][1],
        o[2][0],
        o[2][1],
        o[3][0],
        o[3][1],
        o[4][0],
        o[4][1],
        o[5][0],
        o[5][1],
    )

def calc_marginal_numba(
    cols: dict[str, list[np.ndarray]],
    cols_noncommon: dict[str, list[np.ndarray]],
    domains: dict[str, list[int]],
    x: AttrSelector,
    p: AttrSelectors,
    partial: bool = False,
    zero_fill: float | None = None,
):
    """Calculates the 1 way and 2 way marginals between the subsection of the
    hierarchical attribute x and the attributes p(arents)."""

    l = len(next(iter(cols.values()))[0])
    # Handle parents
    ops = []
    mul = 1
    for attr_name, attr in p.items():
        common = attr.common
        l_mul = 1
        p_partial = partial and attr_name == x.name
        for i, (n, h) in enumerate(attr.cols.items()):
            if common == 0 or i == 0:
                ops.append((mul*l_mul, cols[n][h]))
            else:
                ops.append((mul*l_mul, cols_noncommon[n][h]))

            l_mul *= domains[n][h] - common

        if p_partial:
            mul *= l_mul
        else:
            mul *= l_mul + common
    p_dom = mul

    # Handle x
    common = x.common
    for i, (n, h) in enumerate(x.cols.items()):
        if common == 0 or (i == 0 and not partial):
            ops.append((mul*l_mul, cols[n][h]))
        else:
            ops.append((mul*l_mul, cols_noncommon[n][h]))

        mul *= domains[n][h] - common
    x_dom = mul
    dom = x_dom * p_dom

    # FIXME: partial support
    # Keep only non-common items if there is a parent to source the others
    # if partial:
    #     n = next(iter(x.cols))
    #     _sum_nd = _sum_nd[cols[n][0] >= common]
    #     x_dom = x_dom - x.common

    return numba_inner(dom, l, None, *ops)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_87858/3250864887.py:1 in <cell line: 1>                                           │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_87858/3250864887.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'Op' is not defined

In [15]:
calc_marginal_numba(cols, cols_noncommon, domains, x, p)
%timeit calc_marginal_numba(cols, cols_noncommon, domains, x, p)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_87858/1765432453.py:1 in <cell line: 1>                                           │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_87858/1765432453.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'calc_marginal_numba' is not defined